In [1]:
import pygame
import time
import serial
import re
import numpy as np
from classes.RobotCamera import RobotCamera
from classes.Joystick import Controller
from helpers.process_input import process_input
from helpers.interface import *

pygame 2.5.2 (SDL 2.28.3, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def set_mode(current_mode, new_mode):
    if new_mode == current_mode:
        print("Mode already set to", new_mode)
        return current_mode
    
    current_mode = new_mode
    
    ser.write(b"~\r")
    
    if current_mode == "manual":
        desired_command = "MANUAL MODE!"
        undesired_command = "EXIT"
    elif current_mode == "auto":
        desired_command = "EXIT"
        undesired_command = "MANUAL MODE!"

    check = ''
    start_time = time.time()

    while True:
        check += ser.read_all().decode('ascii')
        print(check)

        if bool(re.search(undesired_command, check)):
            ser.write(b"~\r")
            return new_mode
        if bool(re.search(desired_command, check)):
            return new_mode

        if time.time() - start_time > 10: 
            raise TimeoutError

        time.sleep(0.05)
        

In [3]:
def check_messages(response):
    print(response)
    
    if bool(re.search("DISABLED", response)) or bool(re.search("IMPACT", response)):
        print("ASKJHDS")

    if response != '': print(response)

In [4]:
ROBOT_OPTION_1 = [b"2000\r", b"-12249\r", b"-8210\r", b"-25802\r", b"0\r"]
ROBOT_OPTION_2 = [b"2000\r", b"-13687\r", b"-27171\r", b"-3323\r", b"0\r"]
ROBOT_OPTION_3 = [b"2000\r", b"-1167\r", b"-26033\r", b"13945\r", b"0\r"]

ser = serial.Serial('COM16')
ser.write(b"CON \r")
time.sleep(1)
mode = None
movement_mode = None

mode = set_mode(mode, "auto")

# time.sleep(1)
# print(ser.read_all().decode('ascii'))
# print('mode', mode)

CON 
CONTROL ENABLED.
>
CON 
CONTROL ENABLED.
>
MANUAL MODE!
>
 JOINT M


In [25]:
mode = set_mode(mode, "auto")

print("First movement")

ser.write(b"HERE P1 \r")
time.sleep(0.1)
ser.write(b"SETPV P1 1 -7000\r")
time.sleep(0.5)
ser.write(b"MOVE P1 \r")
time.sleep(0.5)

position_command = ROBOT_OPTION_1
temp_position_command = position_command.copy()
temp_position_command[0] = b'-7000\r'

print("Second movement")

ser.write(b"SETPV P1\r")
time.sleep(0.1)

for command in temp_position_command:
    print(command)
    ser.write(command)
    response = ser.read_all().decode('ascii')
    check_messages(response)
    time.sleep(0.1)

ser.write(b"MOVE P1\r")
time.sleep(2)

print("Third movement")

ser.write(b"SETPV P1\r")
time.sleep(0.1)
for command in position_command:
    print(command)
    ser.write(command)
    response = ser.read_all().decode('ascii')
    check_messages(response)
    time.sleep(0.1)
ser.write(b"MOVE P1 \r")
time.sleep(0.5)

Mode already set to auto
First movement
Second movement
b'-7000\r'
0
 Done.
>MOVE P1 
 Done.
>HERE P1 
 Done.
>SETPV P1 1 -7000
 Done.
>MOVE P1 
 Done.
>SETPV P1
      1 --[-7000] >
0
 Done.
>MOVE P1 
 Done.
>HERE P1 
 Done.
>SETPV P1 1 -7000
 Done.
>MOVE P1 
 Done.
>SETPV P1
      1 --[-7000] >
b'-12249\r'
-7000
      2 --[-12245] >
-7000
      2 --[-12245] >
b'-8210\r'
-12249
      3 --[-8211] >
-12249
      3 --[-8211] >
b'-25802\r'
-8210
      4 --[-25802] >
-8210
      4 --[-25802] >
b'0\r'
-25802
      5 --[0] >
-25802
      5 --[0] >
Third movement
b'2000\r'
0
 Done.
>MOVE P1
 Done.
>SETPV P1
      1 --[-7000] >
0
 Done.
>MOVE P1
 Done.
>SETPV P1
      1 --[-7000] >
b'-12249\r'
2000
      2 --[-12249] >
2000
      2 --[-12249] >
b'-8210\r'
-12249
      3 --[-8210] >
-12249
      3 --[-8210] >
b'-25802\r'
-8210
      4 --[-25802] >
-8210
      4 --[-25802] >
b'0\r'
-25802
      5 --[0] >
-25802
      5 --[0] >


In [9]:
response = ser.read_all().decode('ascii')
print(response)

In [ ]:
response = ser.read_all().decode('ascii')
print(response)

In [ ]:
ser.close()